In [1]:
!python3 -m pip install --upgrade langchain deeplake openai tiktoken

In [2]:
import os
# os.environ["OPENAI_API_KEY"] = "..."

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [4]:
import openai
import os
import IPython
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
print(openai.api_key == None)


False


In [6]:
!pwd

/srv/nfs4/data/openai/Prompt-Engineering-Guide


In [5]:
my_contract = "./docs/dell_eula2023.txt"
my_contract

'./docs/dell_eula2023.txt'

In [8]:
with open(my_contract) as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

Created a chunk of size 1092, which is longer than the specified 1000
Created a chunk of size 1154, which is longer than the specified 1000
Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of size 1684, which is longer than the specified 1000
Created a chunk of size 3121, which is longer than the specified 1000
Created a chunk of size 1016, which is longer than the specified 1000
Created a chunk of size 1265, which is longer than the specified 1000
Created a chunk of size 1123, which is longer than the specified 1000
Created a chunk of size 2861, which is longer than the specified 1000


In [9]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])
# docs = docsearch.similarity_search(query)

Using embedded DuckDB without persistence: data will be transient


Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.

In [10]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [11]:
template_CHINESE = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that Custom-ChatGPT does not know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in CHINESE.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN CHINESE:"""

In [12]:
template_ENGLISH = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that Custom-ChatGPT does not know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in ENGLISH.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN ENGLISH:"""

In [15]:
PROMPT = PromptTemplate(template=template_ENGLISH, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "Where did you get the lease?"
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': "\nI got the lease from the Sample Lease or Rental Agreement provided by Penn State Altoona's Office of Off-Campus Living. It includes the signatures of the landlord and tenant, the property address, the duration of the lease, the monthly rent amount, the utilities and services provided, the deposit amount, the Inventory Move-In Checklist, and the agreement of the parties. \n\nSOURCES: \n1. Sample Lease or Rental Agreement from https://altoona.psu.edu/offices-divisions/student-affairs/off-campus-living/sample-lease-rental-agreement\n2. Signatures and printed names of landlord and tenant (Source 3)"}

In [10]:
query = input("Enter query:")
ans = qa({"query": query})

print("\nCustom-ChatGPT-app answer: " + ans['result'] + "\n\n")
print(ans)


Enter query: who pays for water?



Custom-ChatGPT-app answer:  Tenant pays for water.


{'query': 'who pays for water?', 'result': ' Tenant pays for water.', 'source_documents': [Document(page_content='water\n        garbage collection\n        snow removal\n        other\n    5. Deposit\n    Tenant has paid a deposit of $[deposit amount] of which Landlord acknowledges receipt. Upon regaining possession of the property, Landlord shall refund to Tenant the total amount of the deposit less any damages to the property, normal wear and tear expected, and less any unpaid rent.\n    6. Refund Procedures', metadata={}), Document(page_content='3. Rent\n    Tenant agrees to pay rent in the amount of [monthly rent amount] per month, each payment due on the [day of the month] day of each month and to be made at:\n    [Landlord address]\n    4. Utilities/Services\n    Landlord agrees to provide the utilities and services indicated: [circle all utilities and services that apply]\n        electricity\n        gas\n        water\n   

In [19]:
PROMPT = PromptTemplate(template=template_ENGLISH, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "Who pays the water bill?"
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nThe tenant is responsible for paying the water bill according to the Sample Lease or Rental Agreement from https://altoona.psu.edu/offices-divisions/student-affairs/off-campus-living/sample-lease-rental-agreement. Section 4 of the agreement states that the tenant is responsible for paying for the utilities and services indicated, which includes water. \n\nSOURCES: \nSample Lease or Rental Agreement from https://altoona.psu.edu/offices-divisions/student-affairs/off-campus-living/sample-lease-rental-agreement'}

In [21]:
PROMPT = PromptTemplate(template=template_ENGLISH, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "What happens when the tenant is 11 days late for paying rent"
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nIf the tenant is 11 days late for paying rent, the landlord may retain any portion of the security deposit for damages noted in the Move-Out Checklist to which the Landlord did not object. The tenant may also be subject to other penalties, such as late fees, as outlined in the lease agreement. \n\nSOURCES: \n0. Sample Lease or Rental Agreement from https://altoona.psu.edu/offices-divisions/student-affairs/off-campus-living/sample-lease-rental-agreement\n1. https://www.rocketlawyer.com/form/lease-agreement.rl\n2. https://www.lawdepot.com/contracts/lease-agreement/?loc=US\n3. https://www.thebalancesmb.com/sample-lease-agreement-2866540'}

In [22]:
PROMPT = PromptTemplate(template=template_ENGLISH, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "can tenant sublet the lease?"
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nYes, the tenant can sublet the lease with the written consent of the landlord, which cannot be withheld unreasonably. This is stated in Section 8 of the lease agreement. \n\nSOURCES: \n0. Sample Lease or Rental Agreement from https://altoona.psu.edu/offices-divisions/student-affairs/off-campus-living/sample-lease-rental-agreement\n1. Content from Question\n2. Content from Question\n3. Content from Question'}

In [17]:
PROMPT = PromptTemplate(template=template_CHINESE, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "Who pays the water bill?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n根據第四條，租客同意支付房租，以及提供的其他服務，例如電力、瓦斯、水費、垃圾收集和雪管理等。因此，租客需要支付水費。\n\n來源：\n0. https://altoona.psu.edu/offices-divisions/student-affairs/off-campus-living/sample-lease-rental-agreement\n1. https://www.lawdepot.com/contracts/lease-agreement/?loc=US\n2. https://www.dca.ca.gov/publications/landlordbook/lease-agreements.shtml'}

In [19]:
PROMPT = PromptTemplate(template=template_ENGLISH, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "Who is King Charles?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nCustom-ChatGPT does not know who King Charles is. \n\nSOURCES:\n0. Sample Lease or Rental Agreement from https://altoona.psu.edu/offices-divisions/student-affairs/off-campus-living/sample-lease-rental-agreement\n1. Content: 1. Property\n2. Content: The Landlord may not retain any portion of the Security Deposit for damages noted in the Move-Out Checklist to which the Landlord did not object.\n3. Signatures'}